# REopt API post and polling with localhost or NREL server

## Initialization

In [1]:
%matplotlib notebook
from results_poller import results_plots, reo_optimize, reo_optimize_chpstaging, reo_optimize_localhost, reo_optimize_development
import pandas as pd
import numpy as np
import pickle
import json
import requests
import matplotlib.pyplot as plt
import copy
import os

## Load a .pickle file to get previously saved API response

In [ ]:
save_directory = 'saved_results/'
pickle_file = 'pickle_file_test_1'
with open(save_directory + pickle_file + '.pickle', 'rb') as handle:
    api_response = pickle.load(handle)

## Scenario Inputs (Post), if wanting to do a new API call

In [8]:
#elecloads = list(pd.read_csv(r'C:\Users\dolis\Documents\Projects\Marines ERCIP Japan\Camp Fuji\REopt\fuji_elecprofile.csv', header = 0))
elecloads = list(pd.read_csv(r'C:\Users\dolis\Documents\Projects\Marines ERCIP Japan\Camp Fuji\REopt\fuji_elecprofile-test.csv', header = None).iloc[:,0])
heatloads = list(pd.read_csv(r'C:\Users\dolis\Documents\Projects\Marines ERCIP Japan\Camp Fuji\REopt\load_profile_template_fuji_fuels-test.csv', header = 0).iloc[:,1])
tou_elec_energy = list(pd.read_csv(r'C:\Users\dolis\Documents\Projects\Marines ERCIP Japan\Camp Fuji\REopt\fujiTOU-test.csv', header = None).iloc[:,0])

In [35]:
post_fuji = {
  "Scenario": {
    "timeout_seconds": 400,
    "optimality_tolerance_techs": 0.01,
    "Site": {
      "latitude": 35.31,
      "longitude": 138.87,
      "address": "",
      "land_acres": None,
      "roof_squarefeet": None,
      "elevation_ft": 0.0,
      "Financial": {
        "om_cost_escalation_pct": 0.0,
        #"escalation_pct": 0.0203,
         "escalation_pct": 0.0,
        "boiler_fuel_escalation_pct": 0.0436,
        "chp_fuel_escalation_pct": 0.0436,
        "offtaker_tax_pct": 0.0,
       # "offtaker_discount_pct": 0.030,
        "offtaker_discount_pct": 0.0,
        "third_party_ownership": False,
        "owner_tax_pct": 0.0,
        "owner_discount_pct": 0.0,
        "analysis_years": 1,
        #"owner_discount_pct": 0.04,
        #"analysis_years": 25,
        "value_of_lost_load_us_dollars_per_kwh": 0,
        "microgrid_upgrade_cost_pct": 0.0
      },
      "LoadProfile": {
#        "doe_reference_name": "Hospital",
#        "percent_share": [100.0],
#        "loads_kw": elecloads,
        "loads_kw": [1.0]*8760,
        "year": 2020,
        "loads_kw_is_net": True,
        "critical_loads_kw_is_net": False,
        "outage_start_hour": None,
        "outage_end_hour": None,
        "critical_load_pct": 0.5,
        "outage_is_major_event": True
      },
      "LoadProfileBoilerFuel": {
        "loads_mmbtu_per_hour": [0.0]*8760
#        "loads_mmbtu_per_hour": heatloads
#        "doe_reference_name": "Hospital"
      },
      "ElectricTariff": {
        "urdb_response": {},
        "net_metering_limit_kw": 0.0,
        "interconnection_limit_kw": 100000000.0,
        "chp_standby_rate_us_dollars_per_kw_per_month": 0.0,
        "chp_does_not_reduce_demand_charges": False,
        "add_tou_energy_rates_to_urdb_rate": True,
        "tou_energy_rates_us_dollars_per_kwh": [0.01]*8760 
      },
      "FuelTariff": {
        "existing_boiler_fuel_type": "diesel_oil",
        "boiler_fuel_blended_annual_rates_us_dollars_per_mmbtu": 2.96,
        "chp_fuel_type": "diesel_oil",
        "chp_fuel_blended_annual_rates_us_dollars_per_mmbtu": 2.96
      },
      "PV": {
        "pv_name": "",
        "existing_kw": 0.0,
        "min_kw": 0.0,
        "max_kw": 0.0,
        "macrs_option_years": 0,
        "macrs_bonus_pct": 0.0,
        "macrs_itc_reduction": 0.0,
        "federal_itc_pct": 0.0
      },
      "Storage": {
        "min_kw": 0.0,
        "max_kw": 0,
        "min_kwh": 0.0,
        "max_kwh": 0,
        "macrs_option_years": 0,
        "macrs_bonus_pct": 0.0,
        "macrs_itc_reduction": 0.0
      },
      "CHP": {
        "prime_mover": "recip_engine",
        "size_class": 2,
        "min_kw": 0,
        "max_kw": 0,
        "min_allowable_kw": 0,
        "macrs_option_years": 0,
        "macrs_bonus_pct": 0.0,
        "macrs_itc_reduction": 0.0,
        "federal_itc_pct": 0.0,
        "state_ibi_pct": 0.0,
        "state_ibi_max_us_dollars": 0.0,
        "utility_ibi_pct": 0.0,
        "utility_ibi_max_us_dollars": 0.0,
        "federal_rebate_us_dollars_per_kw": 0.0,
        "state_rebate_us_dollars_per_kw": 0.0,
        "state_rebate_max_us_dollars": 0.0,
        "utility_rebate_us_dollars_per_kw": 0.0,
        "utility_rebate_max_us_dollars": 0.0,
        "pbi_us_dollars_per_kwh": 0.0,
        "pbi_max_us_dollars": 0.0,
        "pbi_years": 0.0,
        "pbi_system_max_kw": 0.0,
        "emissions_factor_lb_CO2_per_mmbtu": 116.9
      },
      "Boiler": {
        "existing_boiler_production_type_steam_or_hw": "steam",
        "boiler_efficiency": 0.75
#        "emissions_factor_lb_CO2_per_mmbtu": 116.9
      }
    },
    "user_uuid": None,
    "description": "",
    "time_steps_per_hour": 1,
    "webtool_uuid": None
  }
}

In [62]:
# Load a json into a python dictionary
# This custom rate tarif was used to get the $15.97 monthly demand and then I added the lookback to make it annual peak demand charge
with open(r'C:\Users\dolis\Documents\Projects\Marines ERCIP Japan\Camp Fuji\REopt\fuji3.json', 'r') as fp:
    URDB1 = json.load(fp)
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"] = URDB1

In [36]:
# Load a json into a python dictionary
# This custom rate tarif was used to get the $15.97 monthly demand and then I added the lookback to make it annual peak demand charge
with open("Fuji3.json", 'r') as fp:
    URDB1 = json.load(fp)
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"] = URDB1
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"]["lookbackMonths"] = [1]*12
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"]["lookbackPercent"] = 1.0
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"]["lookbackRange"] = 0
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"]["flatdemandstructure"] = [[{"rate": 15.97}]]
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"]["flatdemandmonths"] = [0]*12
post_fuji["Scenario"]["Site"]["LoadProfile"]["loads_kw"][3] = 2.0
post_fuji["Scenario"]["Site"]["LoadProfile"]["loads_kw"][8750] = 3.0

In [5]:
post_fuji["Scenario"]["Site"]["ElectricTariff"]["urdb_response"]

{'energyweekdayschedule': [[1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Save the post to a .json file for sharing or future loading in

In [ ]:
# Convert python dictionary post into json and save to a .json file
post_save = api_response['inputs']
with open('response_inputs.json', 'w') as fp:
    json.dump(post_save, fp)

## Or load in a saved .json file for the post

In [2]:
chp_sized_response_dir = r"C:\Bill\AMO_Local\API Benchmarking\Variable Vs Constant CHP Efficiencies\mono_chp_sized_posts"
chp_sized_response = {}
for filename in os.listdir(chp_sized_response_dir):
    chp_sized_response[filename] = json.load(open(os.path.join(chp_sized_response_dir, filename)))

In [ ]:
# Load a json into a python dictionary
with open(r'C:\Django\REopt_Lite_API\reo\tests\posts\delete_cooling.json', 'r') as fp:
    post_1 = json.load(fp)

## Post and poll API to get a new result, if not loading in a previous response
## ...This may take a while!

# Post 1

## use reo_optimize_localhost for local/docker-hosted API
## use reo_optimize_chpstaging for staging-server hosted API

In [38]:
api_response = reo_optimize_localhost(post_fuji)

Response OK from http://localhost:8000/v1/job/?format=json&api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo.
Polling http://localhost:8000/v1/job/be023a16-97bc-4dfe-b1b5-560f46cef948/results/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo for results with interval of 10s...


In [33]:
api_response["outputs"]["Scenario"]["status"]
#api_response["messages"]

'An error occurred. See messages for more.'

In [34]:
api_response['messages']

{'warnings': "{'Default values used for the following:': {'Scenario': 'optimality_tolerance_bau AND add_soc_incentive', 'Scenario>Site': 'outdoor_air_temp_degF', 'Scenario>Site>LoadProfile': 'percent_share', 'Scenario>Site>ElectricTariff': 'add_blended_rates_to_urdb_rate AND wholesale_rate_us_dollars_per_kwh AND wholesale_rate_above_site_load_us_dollars_per_kwh', 'Scenario>Site>FuelTariff': 'boiler_fuel_blended_monthly_rates_us_dollars_per_mmbtu AND chp_fuel_blended_monthly_rates_us_dollars_per_mmbtu', 'Scenario>Site>Wind': 'min_kw AND max_kw AND installed_cost_us_dollars_per_kw AND om_cost_us_dollars_per_kw AND macrs_option_years AND macrs_bonus_pct AND macrs_itc_reduction AND federal_itc_pct AND state_ibi_pct AND state_ibi_max_us_dollars AND utility_ibi_pct AND utility_ibi_max_us_dollars AND federal_rebate_us_dollars_per_kw AND state_rebate_us_dollars_per_kw AND state_rebate_max_us_dollars AND utility_rebate_us_dollars_per_kw AND utility_rebate_max_us_dollars AND pbi_us_dollars_per_k

In [39]:
print(api_response["outputs"]["Scenario"]["Site"]["Financial"]["lcc_us_dollars"])
print(api_response["outputs"]["Scenario"]["Site"]["Financial"]["lcc_bau_us_dollars"])
print(api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs_plus_om_us_dollars"])

663.0
663.0
0.0


In [40]:
print(api_response["outputs"]["Scenario"]["Site"]["Financial"]["lcc_us_dollars"])
print(api_response["outputs"]["Scenario"]["Site"]["Financial"]["lcc_bau_us_dollars"])
print(api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs_plus_om_us_dollars"])

663.0
663.0
0.0


In [68]:
load_year = str(api_response['inputs']['Scenario']['Site']['LoadProfile']['year'])
dti = pd.date_range( load_year +'-01-01', periods=8760, freq='H')
bau_grid_load_less_chiller_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
                                               ['LoadProfile']['loads_kw'], index = dti)
bau_boiler_load_fuel_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
                                         ['LoadProfileBoilerFuel']['loads_mmbtu_per_hour'], index = dti)
boiler_eff = api_response['inputs']['Scenario']['Site']['Boiler']['boiler_efficiency']
bau_grid_load_hrly = bau_grid_load_less_chiller_hrly
bau_grid_load_peak = np.max(bau_grid_load_hrly)
bau_grid_load_mean = np.mean(bau_grid_load_hrly)
bau_grid_load_total = np.sum(bau_grid_load_hrly)
bau_therm_load_peak = np.max(bau_boiler_load_fuel_hrly)*boiler_eff #converted units of fuel to units of heat
bau_therm_load_mean = np.mean(bau_boiler_load_fuel_hrly)*boiler_eff
bau_therm_load_total = np.sum(bau_boiler_load_fuel_hrly)*boiler_eff

print("Elec peak load =", '%.0f' %bau_grid_load_peak, "kW")
print("Elec mean load =", '%.0f' %bau_grid_load_mean, "kW")
print("Elec annual elec =", '%.0f' %bau_grid_load_total, "kWh")
print("Heating peak load =", '%.2f' %bau_therm_load_peak, "MMBtu")
print("Heating mean load =", '%.2f' %bau_therm_load_mean, "MMBtu")


Elec peak load = 1 kW
Elec mean load = 1 kW
Elec annual elec = 8760 kWh
Heating peak load = 0.00 MMBtu
Heating mean load = 0.00 MMBtu


In [3]:
chp_fixed_size_post = {}
for name, post in chp_sized_response.items():
    chp_fixed_size_post[name] = post['inputs'].copy()
    chp_fixed_size_post[name]["Scenario"]["timeout_seconds"] = 300
    chp_fixed_size_post[name]["Scenario"]["optimality_tolerance_techs"] = 0.01
    chp_size = post["outputs"]["Scenario"]["Site"]["CHP"]["size_kw"]
    chp_fixed_size_post[name]["Scenario"]["Site"]["CHP"]["min_kw"] = chp_size
    chp_fixed_size_post[name]["Scenario"]["Site"]["CHP"]["max_kw"] = chp_size
    chp_fixed_size_post[name]["Scenario"]["Site"]["CHP"]["elec_effic_full_load"] = 0.32
    chp_fixed_size_post[name]["Scenario"]["Site"]["CHP"]["elec_effic_half_load"] = 0.28845
    chp_fixed_size_post[name]["Scenario"]["Site"]["CHP"]["thermal_effic_full_load"] = 0.4925
    chp_fixed_size_post[name]["Scenario"]["Site"]["CHP"]["thermal_effic_half_load"] = 0.51573

In [4]:
results_dir = r"C:\Bill\AMO_Local\API Benchmarking\Variable Vs Constant CHP Efficiencies\var_effic_fixed_size_response"
completed_runs = [f.name for f in os.scandir(results_dir)]
for name, post in chp_fixed_size_post.items():
    if name not in completed_runs:
        api_response = reo_optimize_development(post)
        api_response_copy = api_response.copy()
        with open(os.path.join(results_dir, name), "w") as outfile:
            json.dump(api_response_copy, outfile)
        print("Error? ", api_response_copy.get('error'))

C:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Response OK from https://reopt-dev-api1.nrel.gov/v1/job/?format=json&api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo.
Polling https://reopt-dev-api1.nrel.gov/v1/job/bd63200f-a1cf-47cc-be93-cdab58c57710/results/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo for results with interval of 60s...


C:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate ver

SSLError: HTTPSConnectionPool(host='reopt-dev-api1.nrel.gov', port=443): Max retries exceeded with url: /v1/job/bd63200f-a1cf-47cc-be93-cdab58c57710/results/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [ ]:
# Set up a batch run to look at different CHP+PV+Battery cases with diff CHP inputs
iterations = 1
prime_movers = ["recip_engine"] #, "micro_turbine"]
size_class = [2]
part_load = ["constant"] #["default"]
tes_list = ["None", "HotTES", "ColdTES", "Both"]
api_response_dict = {}
for pm in prime_movers:
    for sc in size_class:
        for effic in part_load:
            for tes in tes_list:
                for iters in range(iterations):
                    post = copy.deepcopy(post_1)
                    post["Scenario"]["Site"]["CHP"]["prime_mover"] = pm
                    post["Scenario"]["Site"]["CHP"]["size_class"] = sc
                    if effic == "constant":
                        post["Scenario"]["Site"]["CHP"]["elec_effic_full_load"] = 0.34
                        post["Scenario"]["Site"]["CHP"]["elec_effic_half_load"] = 0.34
                        post["Scenario"]["Site"]["CHP"]["thermal_effic_full_load"] = 0.45
                        post["Scenario"]["Site"]["CHP"]["thermal_effic_half_load"] = 0.45
                    if tes in ["HotTES", "Both"]:
                        #post["Scenario"]["Site"]["HotTES"]["min_gal"] = 30000
                        post["Scenario"]["Site"]["HotTES"]["max_gal"] = 500000
                    if tes in ["ColdTES", "Both"]:
                        #post["Scenario"]["Site"]["ColdTES"]["min_gal"] = 40000
                        post["Scenario"]["Site"]["ColdTES"]["max_gal"] = 500000
                    api_response_dict[pm+"_"+str(sc)+"_"+effic+"_"+tes+"_iter"+str(iters+1)] = reo_optimize_chpstaging(post)
                    print("Error? ", api_response_dict[pm+"_"+str(sc)+"_"+effic+"_"+tes+"_iter"+str(iters+1)]['messages'].get('error'))

In [21]:
techs = ["PV", "Storage", "CHP", "AbsorptionChiller", "HotTES", "ColdTES", "Wind"]
for scenario, resp_dict in api_response_dict.items():
    print(scenario + ":")
    print("Error: ", resp_dict['messages'].get('error'))
    print("REopt Time (sec): ", resp_dict["outputs"]["Scenario"]["Profile"]["reopt_seconds"])
    for t in techs:
        for key, value in resp_dict['outputs']['Scenario']['Site'][t].items():
            if key.startswith("size"):
                print(t + " " + key + " = " + str(resp_dict['outputs']['Scenario']['Site'][t][key]))

NameError: name 'api_response_dict' is not defined

## Start looking at the response/results data

In [22]:
techs = ["PV", "Storage", "CHP", "AbsorptionChiller", "HotTES", "ColdTES", "Wind"]
for t in techs:
    for key, value in api_response['outputs']['Scenario']['Site'][t].items():
        if key.startswith("size"):
            print(t + " " + key + " = " + str(api_response['outputs']['Scenario']['Site'][t][key]))
api_response['outputs']['Scenario']['Profile']

PV size_kw = 0.0
Storage size_kw = 0.0
Storage size_kwh = 0.0
CHP size_kw = 0.0
AbsorptionChiller size_ton = 0.0
HotTES size_gal = 0.0
ColdTES size_gal = 0.0
Wind size_kw = 0.0


{'pre_setup_scenario_seconds': 0.43248963356018066,
 'setup_scenario_seconds': 0.10273456573486328,
 'reopt_seconds': 7.0330283641815186,
 'reopt_bau_seconds': 2.6039845943450928,
 'parse_run_outputs_seconds': 0.3676943778991699,
 'julia_input_construction_seconds': 0.006880044937133789,
 'julia_reopt_preamble_seconds': 0.00012993812561035156,
 'julia_reopt_variables_seconds': 1.2450380325317383,
 'julia_reopt_constriants_seconds': 2.2768969535827637,
 'julia_reopt_optimize_seconds': 0.37321901321411133,
 'julia_reopt_postprocess_seconds': 3.0398640632629395,
 'pyjulia_start_seconds': None,
 'pyjulia_pkg_seconds': None,
 'pyjulia_activate_seconds': None,
 'pyjulia_include_model_seconds': None,
 'pyjulia_make_model_seconds': None,
 'pyjulia_include_reopt_seconds': None,
 'pyjulia_run_reopt_seconds': 7.032681226730347,
 'julia_input_construction_seconds_bau': 0.0049779415130615234,
 'julia_reopt_preamble_seconds_bau': 9.799003601074219e-05,
 'julia_reopt_variables_seconds_bau': 0.6526567

In [ ]:
#api_response['outputs']['Scenario']['Site']['AbsorptionChiller']['size_ton']#['year_one_absorp_chl_thermal_consumption_series_mmbtu_per_hr']
#max(api_response['outputs']['Scenario']['Site']['AbsorptionChiller']['year_one_absorp_chl_thermal_consumption_series_mmbtu_per_hr'])
#max(api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_thermal_production_series_mmbtu_per_hr'])
#max(api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel']['year_one_boiler_thermal_load_series_mmbtu_per_hr'])#['loads_mmbtu_per_hr']))
#max(api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_thermal_production_series_mmbtu_per_hr'])
#api_response['messages']
api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel'].keys()
fuel = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
elec = api_response['outputs']['Scenario']['Site']['CHP']['year_one_electric_energy_produced_kwh']
therm_prod = api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_energy_produced_mmbtu']
therm_waste = sum(['year_one_thermal_to_waste_series_mmbtu_per_hour'])
effic = (elec * 3412 / 1E6 + therm_prod) / fuel
effic_max = (elec * 3412 / 1E6 + therm_prod + therm_waste) / fuel
print("effic = ", effic)
print("effic_max = ", effic_max)
print("therm_util = ", therm_prod / (therm_prod + therm_waste))
# api_response['outputs']['Scenario']['Site']['ElectricTariff']
# api_response['inputs']['Scenario']['Site']['ElectricChiller']

## Save API response and scenario data into a .pickle file for later use

In [ ]:
save_directory = 'saved_results/'
pickle_file = 'api_response_description'
with open(save_directory + pickle_file + '.pickle', 'wb') as handle:
    pickle.dump(api_response, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Extract relevant data and assign object names to them

In [ ]:
#Olis
load_year = str(api_response['inputs']['Scenario']['Site']['LoadProfile']['year'])
dti = pd.date_range( load_year +'-01-01', periods=8760, freq='H')
#bau_grid_load_less_chiller_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
#                                               ['LoadProfile']['loads_kw'], index = dti)
bau_boiler_load_fuel_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
                                         ['LoadProfileBoilerFuel']['loads_mmbtu_per_hour'], index = dti)
# bau_chiller_load_elec_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
#                                           ['LoadProfileChillerElectric']['loads_kw'], index = dti)
boiler_eff = api_response['inputs']['Scenario']['Site']['Boiler']['boiler_efficiency']

bau_grid_load_hrly = bau_grid_load_less_chiller_hrly + bau_chiller_load_elec_hrly
bau_grid_load_peak = np.max(bau_grid_load_hrly)
bau_grid_load_mean = np.mean(bau_grid_load_hrly)
bau_grid_load_total = np.sum(bau_grid_load_hrly)
bau_therm_load_peak = np.max(bau_boiler_load_fuel_hrly)*boiler_eff #converted units of fuel to units of heat
bau_therm_load_mean = np.mean(bau_boiler_load_fuel_hrly)*boiler_eff
bau_therm_load_total = np.sum(bau_boiler_load_fuel_hrly)*boiler_eff
chp_therm_to_elec_full_load = chp_therm_eff_full_load/chp_elec_eff_full_load


chp_elec_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_electric_production_series_kw'], index = dti)
chp_therm_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                      ['year_one_thermal_to_load_series_mmbtu_per_hour'], index = dti)
chp_therm_to_TES_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                     ['year_one_thermal_to_tes_series_mmbtu_per_hour'], index = dti)
chp_therm_total_hrly = chp_therm_to_load_hrly + chp_therm_to_TES_hrly
chp_fuel_annual = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
chp_elec_effic = np.sum(chp_elec_total_hrly)/(chp_fuel_annual * 1E6/3412.0)
chp_total_effic = (np.sum(chp_elec_total_hrly) + np.sum(chp_therm_total_hrly)* 1E6/3412.0)/(chp_fuel_annual * 1E6/3412.0)
chp_elec_cf = np.sum(chp_elec_total_hrly)/(chp_size_kwe*8760)
chp_avg_load_when_on = chp_elec_total_hrly[chp_elec_total_hrly>1].mean()
boiler_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                ['year_one_boiler_fuel_consumption_series_mmbtu_per_hr'], index = dti)
boiler_fuel_annual = boiler_fuel_hrly.resample('Y').sum()

#chp_annual_runhrs = np.count_nonzero(chp_elec_total_hourly)
chp_annual_runhrs = np.sum(chp_elec_total_hrly>1) #alternative to np.count_nonzero to eliminate very small numbers. Sums the number of TRUE
#chp_annual_runhrs = chp_elec_total_hourly[chp_elec_total_hourly>1].count() #alternative to np.count_nonzero to eliminate very small numbers
chp_elec_cf_when_on = chp_avg_load_when_on/chp_size_kwe
chp_annual_heatinghrs = np.sum(chp_therm_total_hrly>0.01) #alternative to np.count_nonzero to eliminate very small numbers
chp_capacity_elec_to_load_peak = chp_size_kwe/bau_grid_load_peak
chp_capacity_elec_to_load_mean = chp_size_kwe/bau_grid_load_mean
chp_capacity_therm_to_load_peak = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_peak*1e6/3412)
chp_capacity_therm_to_load_mean = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_mean*1e6/3412)

load_hours_less_than_chp_capacity = np.sum(bau_grid_load_hrly < chp_size_kwe)

#find base load of bau_grid_load_hrly. Using value with 85% exceedance as described in ASHRAE 'Combined Heat and Power Design Guide' 
base_load_site_elec = bau_grid_load_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]
base_load_site_therm = bau_boiler_load_fuel_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]/boiler_eff
chp_capacity_elec_to_base_load = chp_size_kwe/base_load_site_elec
chp_capacity_therm_to_base_load = chp_size_kwe*chp_therm_to_elec_full_load/(base_load_site_therm*1e6/3412)
chp_gen_elec_to_load = chp_elec_total_hrly.resample('Y').sum()/bau_grid_load_hrly.resample('Y').sum()
chp_gen_therm_to_load = chp_therm_total_hrly.resample('Y').sum()/(bau_boiler_load_fuel_hrly.resample('Y').sum()*boiler_eff)
#net_fuel_consumption_change = 

print("Annual CHP Electric Efficiency =", '%.3f' %chp_elec_effic)
print("Annual CHP Total Efficiency =", '%.3f' %chp_total_effic)
print("Annual CHP Elec Capacity Factor =", '%.2f' %chp_elec_cf)
print("Annual CHP Elec Capacity Factor when running =", '%.2f' %chp_elec_cf_when_on)
print("Annual CHP run hours =", '%.0f' %chp_annual_runhrs, "(fraction",'%.2f'%(chp_annual_runhrs/8760), ")")
print("Annual CHP hours of useful heat =", '%.0f' %chp_annual_heatinghrs, "(fraction",'%.2f'%(chp_annual_heatinghrs/8760), ")")
print("Annual CHP mean power output when on =", '%.0f' %chp_avg_load_when_on, "kW")
print("Elec peak load =", '%.0f' %bau_grid_load_peak, "kW")
print("Elec mean load =", '%.0f' %bau_grid_load_mean, "kW")
print("Elec base load =", '%.0f' %base_load_site_elec, "kW (85% exceedance value)")
print("Heating peak load =", '%.2f' %bau_therm_load_peak, "MMBtu")
print("Heating mean load =", '%.2f' %bau_therm_load_mean, "MMBtu")
print("Heating base load =", '%.2f' %base_load_site_therm, "MMBtu (85% exceedance value)")
print("Ratio of CHP elec capacity to peak elec load =", '%.3f' %chp_capacity_elec_to_load_peak)
print("Ratio of CHP elec capacity to mean elec load =", '%.3f' %chp_capacity_elec_to_load_mean)
print("Ratio of CHP elec capacity to base elec load =", '%.2f' %chp_capacity_elec_to_base_load)
print("Ratio of CHP therm capacity to peak heat load =", '%.3f' %chp_capacity_therm_to_load_peak)
print("Ratio of CHP therm capacity to mean heat load =", '%.3f' %chp_capacity_therm_to_load_mean)
print("Ratio of CHP therm capacity to base heat load =", '%.3f' %chp_capacity_therm_to_base_load)
print("Useful heat recovery ratio =", '%.3f' )
print("Annual fraction of site elec load served by CHP =", '%.3f' %chp_gen_elec_to_load.iloc[0][0])
print("Annual fraction of site thermal load served by CHP =", '%.3f' %chp_gen_therm_to_load.iloc[0][0])


## ===============================================================

## Example from Josiah for how to setup batch API calls

## Comparing DOE Reference Buildings

In [ ]:
doe_list = ['RetailStore', 'StripMall', 'Warehouse', 'Supermarket']

In [ ]:
api_response_dict = dict()

for name in doe_list:
    newpost = post
    newpost['Scenario']['Site']["LoadProfile"]["doe_reference_name"] = name
    
    print('Beginning Scenario:', name)
    api_response = reo_optimize_localhost(newpost)
    api_response_dict[name] = api_response
    
    if api_response['outputs']['Scenario']['status'] != 'optimal':
        print('non-optimial solution')

In [ ]:
results_dict['RetailStore']['outputs']